In [ ]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms

# 读取数据集

In [ ]:
trans = transforms.ToTensor()  # 这里定义了一个图像转换操作 trans，它会将输入的 PIL 图像转换为 PyTorch 的 Tensor 类型，同时将图像的像素值从 0 到 255 归一化到 0 到 1 之间，并转换为 32 位浮点数格式。

mnist_train = torchvision.datasets.FashionMNIST(root="../../data", train=True, transform=trans, download=True)  # 加载了 FashionMNIST 数据集的训练部分
mnist_test = torchvision.datasets.FashionMNIST(root="../../data", train=False, transform=trans, download=True)  # 加载测试集加载了 FashionMNIST 数据集的测试部分，参数解释与加载训练集时相同，只是 train=False 表示加载测试集数据。

FashionMNIST 数据集是一个由 Zalando 公司发布的图像数据集，用于替代经典的 MNIST 数据集。MNIST 数据集包含手写数字的图像，而 FashionMNIST 数据集包含的是各种服装商品的图像。这个数据集被广泛用于机器学习和计算机视觉的研究与教学。

### FashionMNIST 数据集的特点：

1. **数据量**：
   - 包含 60,000 张训练图像和 10,000 张测试图像。
   
2. **图像尺寸**：
   - 每张图像的尺寸为 28x28 像素，灰度图像。
   
3. **类别**：
   - 数据集包含 10 个类别，每个类别代表一种服装商品：
     1. T恤/上衣
     2. 裤子
     3. 套头衫
     4. 连衣裙
     5. 外套
     6. 凉鞋
     7. 衬衫
     8. 运动鞋
     9. 包
     10. 短靴

4. **标签**：
   - 每张图像都有一个对应的标签，表示图像所属的类别。


In [ ]:
len(mnist_train), len(mnist_test)

Fashion-MNIST由10个类别的图像组成， 每个类别由训练数据集（train dataset）中的6000张图像 和测试数据集（test dataset）中的1000张图像组成。 因此，训练集和测试集分别包含60000和10000张图像。 测试数据集不会用于训练，只用于评估模型性能。

In [ ]:
mnist_train[0][0].shape

每个输入图像的高度和宽度均为28像素。 数据集由灰度图像组成，其通道数为1。 

`mnist_train` 是一个包含 FashionMNIST 数据集的 PyTorch 数据集对象（`Dataset`）。具体来说，它是一个 `torchvision.datasets.FashionMNIST` 对象。这个对象是 PyTorch 提供的一种数据集抽象，用于加载和处理常见的数据集。

1. `mnist_train[0]`：
    - 返回数据集中的第一个样本。
    - FashionMNIST 数据集的每个样本是一个元组 `(image, label)`。
    - `image` 是一个图像的张量。
    - `label` 是该图像的标签，表示图像所属的类别。

2. `mnist_train[0][0]`：
    - 返回第一个样本的图像部分（`image`）。
    - 这种访问方式让我们可以直接获取图像数据。


In [ ]:
# Fashion-MNIST中包含的10个类别，分别为t-shirt（T恤）、trouser（裤子）、pullover（套衫）、dress（连衣裙）、coat（外套）、sandal（凉鞋）、shirt（衬衫）、sneaker（运动鞋）、bag（包）和ankle boot（短靴）。 以下函数用于在数字标签索引及其文本名称之间进行转换。
def get_fashion_mnist_labels(labels):  #@save
    """返回Fashion-MNIST数据集的文本标签"""
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [ ]:
from d2l import torch as d2l
d2l.use_svg_display()

# 创建一个函数来可视化这些样本。
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """绘制图像列表"""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # 图片张量
            ax.imshow(img.numpy())
        else:
            # PIL图片
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [ ]:
X, y = next(iter(data.DataLoader(mnist_train, batch_size=18)))
show_images(X.reshape(18, 28, 28), 2, 9, titles=get_fashion_mnist_labels(y));

# 读取小批量

为了使我们在读取训练集和测试集时更容易，我们使用内置的数据迭代器，而不是从零开始创建。 回顾一下，在每次迭代中，数据加载器每次都会读取一小批量数据，大小为batch_size。 通过内置数据迭代器，我们可以随机打乱了所有样本，从而无偏见地读取小批量。

In [ ]:
batch_size = 256

def get_dataloader_workers():  #@save
    """使用4个进程来读取数据"""
    return 4

train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True,
                             num_workers=get_dataloader_workers())

In [ ]:
timer = d2l.Timer()
for X, y in train_iter:
    continue
f'{timer.stop():.2f} sec'
# 读取训练数据所需的时间。

# 整合所有组件

现在我们定义load_data_fashion_mnist函数，用于获取和读取Fashion-MNIST数据集。 这个函数返回训练集和验证集的数据迭代器。 此外，这个函数还接受一个可选参数resize，用来将图像大小调整为另一种形状。

In [ ]:
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=get_dataloader_workers()))

下面，我们通过指定resize参数来测试load_data_fashion_mnist函数的图像大小调整功能。

In [ ]:
train_iter, test_iter = load_data_fashion_mnist(32, resize=64)
for X, y in train_iter:
    print(X.shape, X.dtype, y.shape, y.dtype)
    break